## File ingestion and schema validation

### 1. Import libraries

In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import dask.dataframe as dd
import modin.pandas as mpd
import os
import yaml
import pandas as pd


### 2. Check computational efficiencies for different file reading modules

Dataset: Data on Traffic and Pedestrian Stops by Police in California
(https://www.kaggle.com/datasets/stanford-open-policing/stanford-open-policing-project-california/)

In [28]:
# pandas
start_time = time.time()
df_pandas = pd.read_csv('CA_2013_onwards.csv')
pandas_time = time.time() - start_time

In [64]:
# dask
start_time = time.time()
df_dask = dd.read_csv('CA_2013_onwards.csv')
dask_time = time.time() - start_time

In [30]:
# modin.pandas
start_time = time.time()
df_modin = mpd.read_csv('CA_2013_onwards.csv')
modin_time = time.time() - start_time

(raylet) Spilled 2114 MiB, 12 objects, write throughput 453 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


In [31]:
print('Computational efficiency of pandas: ', pandas_time)
print('Computational efficiency of dask: ', dask_time)
print('Computational efficiency of modin: ', modin_time)
print('')
print('Dask shows the fastest efficiency for large dataset loading.')

Computational efficiency of pandas:  63.183443784713745
Computational efficiency of dask:  0.04871702194213867
Computational efficiency of modin:  41.92890810966492

Dask shows the fastest efficiency for large dataset loading.


## 3. Basic Validation on Data Columns

#### 3-1. Checking types

In [32]:
print(df_pandas.shape)
df_pandas.head(5)

(14536338, 23)


,id,state,stop_date,stop_time,location_raw,county_name,county_fips,fine_grained_location,police_department,driver_gender,...,driver_race,violation_raw,violation,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,ethnicity
0,CA-2013-0000001,CA,2013-01-01,NaN,San Diego,San Diego County,6073.0,NaN,NaN,M,...,Hispanic,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,NaN,False,CHP 215,False,H
1,CA-2013-0000002,CA,2013-01-01,NaN,San Diego,San Diego County,6073.0,NaN,NaN,F,...,Black,Moving Violation (VC),Moving violation,False,No Search,NaN,False,CHP 215,False,B
2,CA-2013-0000003,CA,2013-01-01,NaN,San Diego,San Diego County,6073.0,NaN,NaN,M,...,White,Moving Violation (VC),Moving violation,False,No Search,NaN,False,CHP 215,False,W
3,CA-2013-0000004,CA,2013-01-01,NaN,San Diego,San Diego County,6073.0,NaN,NaN,M,...,White,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,NaN,False,CHP 215,False,W
4,CA-2013-0000005,CA,2013-01-01,NaN,San Diego,San Diego County,6073.0,NaN,NaN,M,...,Black,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,NaN,False,CHP 215,False,B


In [33]:
df_pandas.dtypes

id                        object
state                     object
stop_date                 object
stop_time                float64
location_raw              object
county_name               object
county_fips              float64
fine_grained_location    float64
police_department        float64
driver_gender             object
driver_age_raw            object
driver_age               float64
driver_race_raw           object
driver_race               object
violation_raw             object
violation                 object
search_conducted            bool
search_type_raw           object
search_type               object
contraband_found          object
stop_outcome              object
is_arrested                 bool
ethnicity                 object
dtype: object

In [34]:
df_pandas.describe()

,stop_time,county_fips,fine_grained_location,police_department,driver_age
count,0.0,1.346002e+07,0.0,0.0,0.0
mean,NaN,6.053165e+03,NaN,NaN,NaN
std,NaN,2.738416e+01,NaN,NaN,NaN
min,NaN,6.001000e+03,NaN,NaN,NaN
25%,NaN,6.037000e+03,NaN,NaN,NaN
50%,NaN,6.057000e+03,NaN,NaN,NaN
75%,NaN,6.073000e+03,NaN,NaN,NaN
max,NaN,6.113000e+03,NaN,NaN,NaN


#### 3-2. Data Cleaning

In [50]:
# drop the columns above with 'NaN'
df = df_pandas.copy()
df = df.drop(['stop_time','fine_grained_location', 'police_department','driver_age'], axis=1)
df.head(5)

,id,state,stop_date,location_raw,county_name,county_fips,driver_gender,driver_age_raw,driver_race_raw,driver_race,violation_raw,violation,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,ethnicity
0,CA-2013-0000001,CA,2013-01-01,San Diego,San Diego County,6073.0,M,25-32,Hispanic,Hispanic,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,NaN,False,CHP 215,False,H
1,CA-2013-0000002,CA,2013-01-01,San Diego,San Diego County,6073.0,F,33-39,Black,Black,Moving Violation (VC),Moving violation,False,No Search,NaN,False,CHP 215,False,B
2,CA-2013-0000003,CA,2013-01-01,San Diego,San Diego County,6073.0,M,25-32,White,White,Moving Violation (VC),Moving violation,False,No Search,NaN,False,CHP 215,False,W
3,CA-2013-0000004,CA,2013-01-01,San Diego,San Diego County,6073.0,M,49+,White,White,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,NaN,False,CHP 215,False,W
4,CA-2013-0000005,CA,2013-01-01,San Diego,San Diego County,6073.0,M,40-48,Black,Black,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,NaN,False,CHP 215,False,B


In [51]:
df = df.drop(['search_type'], axis=1)
df.head(5)

,id,state,stop_date,location_raw,county_name,county_fips,driver_gender,driver_age_raw,driver_race_raw,driver_race,violation_raw,violation,search_conducted,search_type_raw,contraband_found,stop_outcome,is_arrested,ethnicity
0,CA-2013-0000001,CA,2013-01-01,San Diego,San Diego County,6073.0,M,25-32,Hispanic,Hispanic,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,False,CHP 215,False,H
1,CA-2013-0000002,CA,2013-01-01,San Diego,San Diego County,6073.0,F,33-39,Black,Black,Moving Violation (VC),Moving violation,False,No Search,False,CHP 215,False,B
2,CA-2013-0000003,CA,2013-01-01,San Diego,San Diego County,6073.0,M,25-32,White,White,Moving Violation (VC),Moving violation,False,No Search,False,CHP 215,False,W
3,CA-2013-0000004,CA,2013-01-01,San Diego,San Diego County,6073.0,M,49+,White,White,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,False,CHP 215,False,W
4,CA-2013-0000005,CA,2013-01-01,San Diego,San Diego County,6073.0,M,40-48,Black,Black,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,False,CHP 215,False,B


In [52]:
df.columns

Index(['id', 'state', 'stop_date', 'location_raw', 'county_name',
       'county_fips', 'driver_gender', 'driver_age_raw', 'driver_race_raw',
       'driver_race', 'violation_raw', 'violation', 'search_conducted',
       'search_type_raw', 'contraband_found', 'stop_outcome', 'is_arrested',
       'ethnicity'],
      dtype='object')

#### 3-3. Changing formats

In [56]:
df.dtypes

id                               object
state                            object
stop_date                        object
location_raw                     object
county_name                      object
county_fips                     float64
driver_gender                    object
driver_age_raw                   object
driver_race_raw                  object
driver_race                      object
violation_raw                    object
violation                        object
search_conducted                   bool
search_type_raw                  object
contraband_found                 object
stop_outcome                     object
is_arrested                        bool
ethnicity                        object
stop_date2          datetime64[ns, UTC]
dtype: object

In [59]:
# change date format from object to date
df['stop_date'] = pd.to_datetime(df['stop_date'], utc=True).dt.strftime('%m/%d/%Y')
df.head(5)

,id,state,stop_date,location_raw,county_name,county_fips,driver_gender,driver_age_raw,driver_race_raw,driver_race,violation_raw,violation,search_conducted,search_type_raw,contraband_found,stop_outcome,is_arrested,ethnicity,stop_date2
0,CA-2013-0000001,CA,01/01/2013,San Diego,San Diego County,6073.0,M,25-32,Hispanic,Hispanic,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,False,CHP 215,False,H,01/01/2013
1,CA-2013-0000002,CA,01/01/2013,San Diego,San Diego County,6073.0,F,33-39,Black,Black,Moving Violation (VC),Moving violation,False,No Search,False,CHP 215,False,B,01/01/2013
2,CA-2013-0000003,CA,01/01/2013,San Diego,San Diego County,6073.0,M,25-32,White,White,Moving Violation (VC),Moving violation,False,No Search,False,CHP 215,False,W,01/01/2013
3,CA-2013-0000004,CA,01/01/2013,San Diego,San Diego County,6073.0,M,49+,White,White,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,False,CHP 215,False,W,01/01/2013
4,CA-2013-0000005,CA,01/01/2013,San Diego,San Diego County,6073.0,M,40-48,Black,Black,Mechanical or Nonmoving Violation (VC),Equipment,False,No Search,False,CHP 215,False,B,01/01/2013


#### 3-4. Checking duplicated rows

In [62]:
dupli = df.duplicated(subset=df.columns.tolist(), keep=False)
duplicated_rows = df[dupli]
duplicated_rows

,id,state,stop_date,location_raw,county_name,county_fips,driver_gender,driver_age_raw,driver_race_raw,driver_race,violation_raw,violation,search_conducted,search_type_raw,contraband_found,stop_outcome,is_arrested,ethnicity,stop_date2


#### 3-5. Check missing or null values

In [67]:
df.isnull().sum()

id                        0
state                     0
stop_date                 0
location_raw              4
county_name         1076319
county_fips         1076319
driver_gender             0
driver_age_raw            0
driver_race_raw           0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type_raw           0
contraband_found     456292
stop_outcome              0
is_arrested               0
ethnicity                 0
stop_date2                0
dtype: int64

In [96]:
df.shape

(14536338, 19)

In [98]:
# size of the file
file_stats = os.stat('CA_2013_onwards.csv')
file_stats.st_size

2493849902

### 4. Write YAML file

In [94]:
%%writefile file.yaml
file_type: csv
dataset_name: CA_2013_onwards
file_name: ca_2013_onwords
table_name: df
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - id
    - state
    - stop_date
    - location_raw
    - county_name
    - county_fips
    - driver_gender
    - driver_age_raw
    - driver_race_raw
    - driver_race
    - violation_raw
    - violation
    - search_conducted
    - search_type_raw
    - contraband_found
    - stop_outcome
    - is_arrested
    - ethnicity

Writing file.yaml


### 5. Write the file in gz format

In [95]:
df.to_csv('ca_2013_onwords.csv.gz', sep='|', compression='gzip', index=False)